In [1]:
import pandas as pd
import tweepy
import numpy as np
pd.options.display.max_colwidth=100

In [2]:
import yaml

In [3]:
with open("access_config.yaml") as f:
    access_dict = yaml.safe_load(f)

In [4]:
access_dict

{'api_key': 'RjJ5L8qguRiY0kwjCFhWyHP3j',
 'api_key_secret': 'MPqXB2ZJV4kinSyDotYfj5zcK2eaipPZsIiNCY6gq6B6DAU8p3',
 'access_token': '1390221257384333312-Kr0yidWBDuE0Ptw85AjMbNR16eIOVY',
 'access_token_secret': 'RSADZsVBOJ7hPpE0mOT3CwDumSAirpJTlVshl3pK7AYqr',
 'bearer_token': 'AAAAAAAAAAAAAAAAAAAAAFKljwEAAAAA8yhItLXWJbmKjOBmgbl4hD5H1Ks%3Deqdo0MwOJ0lV1mtjRcydp4Mx4Er3XhZvxYmuqXwkIfC6iseO8T'}

In [5]:
# authentication
api_key = access_dict['api_key']
api_key_secret = access_dict['api_key_secret']
access_token = access_dict['access_token']
access_token_secret = access_dict['access_token_secret']
bearer_token = access_dict['bearer_token']

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re


st = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))


def clean_text(x, st, sw):
    x = x.lower()
    x = re.sub('[^a-zA-Z]', ' ', x)
    x = ' '.join(st.stem(text) for text in x.split() if text not in sw)

    return x.split(' ')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lsaw9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lsaw9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
v2_client = tweepy.Client(bearer_token=bearer_token, consumer_key=api_key, consumer_secret=api_key_secret, access_token=access_token, access_token_secret=access_token_secret)

In [8]:
query = '#twitter #elonmusk lang:en -is:retweet'

In [9]:
more_tweets = tweepy.Paginator(v2_client.search_recent_tweets,
                 query=query,
                 max_results = 100,
                 start_time='2022-12-05T00:00:00.000Z',
                 end_time='2022-12-09T23:59:59.000Z',
                 tweet_fields = ['author_id','created_at','text','source','lang','geo'],
                 user_fields = ['name','username','location','verified'],
                 expansions = ['geo.place_id', 'author_id'],
                 place_fields = ['country','country_code']).flatten(limit=10000)

In [10]:
more_data = list()
for tweet in more_tweets:
    more_data.append(
        [tweet.author_id, tweet.created_at, tweet.id, tweet.text]
    )

df = pd.DataFrame(data=more_data,
                  columns=['author_id', 'created_at', 'id', 'text']
)

In [11]:
df.shape

(2635, 4)

In [12]:
df['text'] = df['text'].apply(lambda x: clean_text(x, st, stop_words))

In [13]:
df['timestamp'] = pd.to_datetime(df['created_at']).dt.tz_localize(None)
df['dec_hour'] = df['timestamp'].apply(lambda x: x.day * 24 + x.hour)
df['time_windw'] = df['dec_hour'] // 6

In [14]:
df

,author_id,created_at,id,text,timestamp,dec_hour,time_windw
0,1357913221898862592,2022-12-09 23:57:46+00:00,1601365518711144449,"[janitor, laid, twitter, headquart, said, one, elonmusk, team, member, told, replac, robot, news...",2022-12-09 23:57:46,239,39
1,1345223352122748928,2022-12-09 23:57:03+00:00,1601365337542635520,"[k, rat, twitter, gate, break, news, twitter, twitterg, elonmusk, breakingnew, https, co, grt, c...",2022-12-09 23:57:03,239,39
2,20554879,2022-12-09 23:56:51+00:00,1601365287399723008,"[rose, webster, getmygist, dear, eltonjohn, eltonoffici, agre, elonmusk, elonmusk, still, suspen...",2022-12-09 23:56:51,239,39
3,1345223352122748928,2022-12-09 23:56:43+00:00,1601365257548890113,"[ashlynn, twitter, gate, break, news, twitter, twitterg, elonmusk, breakingnew, https, co, grt, ...",2022-12-09 23:56:43,239,39
4,1345223352122748928,2022-12-09 23:55:48+00:00,1601365024374919168,"[dirtydalerz, abolishthedea, kayodoherti, twitter, gate, break, news, twitter, twitterg, elonmus...",2022-12-09 23:55:48,239,39
...,...,...,...,...,...,...,...
2630,699632144168128513,2022-12-05 00:12:29+00:00,1599557284497170432,"[tomorrow, drop, smoke, br, kz, tune, monday, pm, est, blackguytv, elonmusk, twitter, https, co,...",2022-12-05 00:12:29,120,20
2631,137141050,2022-12-05 00:12:06+00:00,1599557188166352896,"[elon, eventu, decid, crypto, use, payment, twitter, like, includ, doge, bitcoin, altcoin, might...",2022-12-05 00:12:06,120,20
2632,1355073655357714435,2022-12-05 00:11:45+00:00,1599557099734073345,"[elonmusk, hand, bot, need, purg, twitter, twitterfil, twitterg, elonmusk, bidenworstpresidentin...",2022-12-05 00:11:45,120,20
2633,1355073655357714435,2022-12-05 00:11:32+00:00,1599557046747090944,"[elonmusk, found, hand, bot, need, purg, twitter, twitterfil, twitterg, elonmusk, bidenworstpres...",2022-12-05 00:11:32,120,20


In [215]:
df.to_csv('biden_tweets_raw_29_11.csv', index=False)

In [221]:
a = '2022-11-30T00:00:00.000Z'
a.split('T')[0]

'2022-11-30'

In [25]:
df = pd.read_csv('musk_twitter_2022-12-10.csv')

In [37]:
df[['text', 'time_window']].to_records()[0]

(0, "['twitter', 'delet', 'billion', 'account', 'lol', 'elonmusk', 'waiv', 'due', 'dilig', 'billion', 'twitter', 'purchas', 'https', 'co', 'btvdaeft']", 43)

In [38]:
from collections import defaultdict
dd = defaultdict(lambda: list())

In [40]:
for record in df[['text', 'time_window']].to_records():
    dd[record[2]].append(eval(record[1]))

In [57]:
seq = list()
for k,v in dd.items():
    seq.append(v)

In [59]:
len(seq)

4